# <img src="ucsur-logo.png" alt="Logo UCSUR" width=200 hight=400 align="right">


<br><br><br>
<h1><font color="#1D65DD" size=5>Programación en Data Science</font></h1>



<h1><font color="#1D65DD" size=6>Módulo II: Pandas III</font></h1>

<br>
<div style="text-align: right">
<font color="#1D65DD" size=3>Yuri Coicca, M.Sc.</font><br>
<font color="#1D65DD" size=3>Universidad Científica del Sur</font><br>
<font color="#1D65DD" size=3>Maestria en Ciencia de Datos</font><br>
</div>

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [9. Multi-índices](#section9)
* [10. Combinación de DataFrames](#section10)
    * [<font face="monospace">append()</font>](#section101)
    * [<font face="monospace">concat()</font>](#section102)
    * [<font face="monospace">merge()</font>](#section103)
    * [<font face="monospace">join()</font>](#section104)
    
* [11. Tablas dinámicas y reestructuración](#section11)
    * [Creación de tablas dinámicas](#section111)
    * [Reestructuración de tablas](#section112) 

In [1]:
# Permite ajustar la anchura de la parte útil de la libreta (reduce los márgenes)
from IPython.core.display import display, HTML
display(HTML("<style>.container{ width:98%}</style>"))

<a id="section9"></a> 
# <font color="#004D7F">9. Multi-índices </font>
<br>

_Pandas_ permite utilizar varios niveles de indexación, tanto para filas como para columnas. En este tutorial se describen los conceptos necesarios para el uso más común de estas estructuras. Se puede encontrar más información al respecto en la ([documentación](https://pandas.pydata.org/pandas-docs/stable/advanced.html)). 


Cuando se proporcionan _varias colecciones_ como índice en la construcción del _DataFrame_, se crea un multi-indice. 

In [2]:
import pandas as pd
import numpy as np

compras = [('Álvaro', 'Queso', 22.50),
           ('Benito', 'Vino', 14.50),
           ('Fernando', 'Jamón', 50.00),
           ('Martín', 'Aceite', 20.00),
           ('Hernán', 'Azafrán', 3.00)]

df_compras = pd.DataFrame(compras, 
                          columns=['Nombre','Producto','Precio'],
                            index=[['Tienda 1', 'Tienda 1', 'Tienda 2', 'Tienda 3', 'Tienda 3'],
                                   ['Albacete', 'Albacete', 'Villarrobledo', 'Tomelloso', 'Alcázar']])
df_compras

Nombre Producto  Precio
Tienda 1 Albacete         Álvaro    Queso    22.5
         Albacete         Benito     Vino    14.5
Tienda 2 Villarrobledo  Fernando    Jamón    50.0
Tienda 3 Tomelloso        Martín   Aceite    20.0
         Alcázar          Hernán  Azafrán     3.0

El índice se divide en niveles organizados de __manera jerárquica__. 

In [3]:
df_compras.index

MultiIndex([('Tienda 1',      'Albacete'),
            ('Tienda 1',      'Albacete'),
            ('Tienda 2', 'Villarrobledo'),
            ('Tienda 3',     'Tomelloso'),
            ('Tienda 3',       'Alcázar')],
           )

La función `swaplevel` permite cambiar la jerarquía de índices. 

In [7]:
df_compras.swaplevel(i=1,j=0)
#df_compras.swaplevel(i=1,j=0, axis=0) #quivalente

Nombre Producto  Precio
Albacete      Tienda 1    Álvaro    Queso    22.5
              Tienda 1    Benito     Vino    14.5
Villarrobledo Tienda 2  Fernando    Jamón    50.0
Tomelloso     Tienda 3    Martín   Aceite    20.0
Alcázar       Tienda 3    Hernán  Azafrán     3.0

También se puede establecer un multi-índice mediante `set_index()`. La siguiente celda de código lee un conjunto de datos y establece un índice con un nivel principal, `state`, y otro secundario, `name`. La jerarquía corresponde al orden en que aparecen las columnas.

In [8]:
df = pd.read_csv('data/county.txt', sep='\t')
df.set_index(['state','name'], inplace=True)
df.head()

pop2000  pop2010  fed_spend  poverty  homeownership  \
state   name                                                                   
Alabama Autauga County   43671.0    54571   6.068095     10.6           77.5   
        Baldwin County  140415.0   182265   6.139862     12.2           76.7   
        Barbour County   29038.0    27457   8.752158     25.0           68.0   
        Bibb County      20826.0    22915   7.122016     12.6           82.9   
        Blount County    51024.0    57322   5.130910     13.4           82.0   

                        multiunit  income  med_income  
state   name                                           
Alabama Autauga County        7.2   24568       53255  
        Baldwin County       22.6   26469       50147  
        Barbour County       11.1   15875       33219  
        Bibb County           6.6   19918       41770  
        Blount County         3.7   21070       45549

Cuando se utiliza un índice a varios niveles, el acceso natural a filas se hace mediante tuplas cuyo tamaño corresponde al número de índices, y con los valores del índice en cada nivel.

In [11]:
#df.loc[('Alabama','Bibb County')]          # Accede a una fila
df.loc['Alabama','Bibb County']             # Estas dos notaciones son equivalentes

pop2000          20826.000000
pop2010          22915.000000
fed_spend            7.122016
poverty             12.600000
homeownership       82.900000
multiunit            6.600000
income           19918.000000
med_income       41770.000000
Name: (Alabama, Bibb County), dtype: float64

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Hablamos de varios niveles porque pueden ser más de dos (aunque rara vez se da esta circunstancia). 
</div>

No es necesario especificar valores en todos los niveles para localizar elementos. Es posible omitir los valores ___a partir de un nivel___. 

In [12]:
df.loc['Alabama'].head()

,pop2000,pop2010,fed_spend,poverty,homeownership,multiunit,income,med_income
name,,,,,,,,
Autauga County,43671.0,54571,6.068095,10.6,77.5,7.2,24568,53255
Baldwin County,140415.0,182265,6.139862,12.2,76.7,22.6,26469,50147
Barbour County,29038.0,27457,8.752158,25.0,68.0,11.1,15875,33219
Bibb County,20826.0,22915,7.122016,12.6,82.9,6.6,19918,41770
Blount County,51024.0,57322,5.130910,13.4,82.0,3.7,21070,45549


Este tipo de indexación, mediante el valor en el índice principal, permite _slicing_.

In [13]:
df.loc['Alabama':'California'].tail()

pop2000  pop2010  fed_spend  poverty  \
state      name                                                     
California Tulare County    368021.0   442179   5.419988     22.9   
           Tuolumne County   54501.0    55365   8.138264     11.7   
           Ventura County   753197.0   823318   7.379247      9.2   
           Yolo County      168660.0   200849  11.132796     17.1   
           Yuba County       60219.0    72155  12.109237     20.0   

                            homeownership  multiunit  income  med_income  
state      name                                                           
California Tulare County             59.3       14.4   17966       43851  
           Tuolumne County           70.2        8.6   25483       47462  
           Ventura County            66.4       20.2   32348       75348  
           Yolo County               54.1       30.6   27420       57077  
           Yuba County               59.8       17.8   19937       46807

También es posible utilizar multi-índices en las columnas. El siguiente código crea un multi-índice y lo establece en el `DataFrame` anterior.

In [14]:
df.columns

Index(['pop2000', 'pop2010', 'fed_spend', 'poverty', 'homeownership',
       'multiunit', 'income', 'med_income'],
      dtype='object')

El multi-índice se puede crear también a partir de una lista de tuplas. 

In [15]:
# Crea la lista de tuplas
level1 = ['population', 'population', 'money', 'money', 'money','money','money','money']
level2 = df.columns
tuples = list(zip(level1,level2))
tuples

[('population', 'pop2000'),
 ('population', 'pop2010'),
 ('money', 'fed_spend'),
 ('money', 'poverty'),
 ('money', 'homeownership'),
 ('money', 'multiunit'),
 ('money', 'income'),
 ('money', 'med_income')]

In [16]:
# Crea el multi-índice
m_columns = pd.MultiIndex.from_tuples(tuples, names=['principal', 'secundario'])
m_columns

MultiIndex([('population',       'pop2000'),
            ('population',       'pop2010'),
            (     'money',     'fed_spend'),
            (     'money',       'poverty'),
            (     'money', 'homeownership'),
            (     'money',     'multiunit'),
            (     'money',        'income'),
            (     'money',    'med_income')],
           names=['principal', 'secundario'])

In [17]:
# Establece un multi-índice como índice de columnas.
df.columns = m_columns
df.head()

principal              population             money                        \
secundario                pop2000 pop2010 fed_spend poverty homeownership   
state   name                                                                
Alabama Autauga County    43671.0   54571  6.068095    10.6          77.5   
        Baldwin County   140415.0  182265  6.139862    12.2          76.7   
        Barbour County    29038.0   27457  8.752158    25.0          68.0   
        Bibb County       20826.0   22915  7.122016    12.6          82.9   
        Blount County     51024.0   57322  5.130910    13.4          82.0   

principal                                           
secundario             multiunit income med_income  
state   name                                        
Alabama Autauga County       7.2  24568      53255  
        Baldwin County      22.6  26469      50147  
        Barbour County      11.1  15875      33219  
        Bibb County          6.6  19918      41770  
        Blount County        3.7  21070      45549

<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> Existen varios métodos que permiten crear un multiíndice [(enlace)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html).
</div>



Al igual que con las filas, se puede acceder a las columnas individualmente, mediante tuplas. 

In [18]:
df['population','pop2000'].head()

state    name          
Alabama  Autauga County     43671.0
         Baldwin County    140415.0
         Barbour County     29038.0
         Bibb County        20826.0
         Blount County      51024.0
Name: (population, pop2000), dtype: float64

También se pueden omitir los valores del índice a partir de un nivel.

In [19]:
df['population'].head()

secundario               pop2000  pop2010
state   name                             
Alabama Autauga County   43671.0    54571
        Baldwin County  140415.0   182265
        Barbour County   29038.0    27457
        Bibb County      20826.0    22915
        Blount County    51024.0    57322

Puede accederse a los elementos mediante el uso de varias (en este caso dos) tuplas. 

In [21]:
df.loc[('Alabama','Autauga County'), ('population','pop2010')]

54571.0

También pueden accederse a los elementos  a través del nivel (o niveles) más bajos del índice. 

In [22]:
df.loc['Alabama', 'money'].head()

secundario,fed_spend,poverty,homeownership,multiunit,income,med_income
name,,,,,,
Autauga County,6.068095,10.6,77.5,7.2,24568,53255
Baldwin County,6.139862,12.2,76.7,22.6,26469,50147
Barbour County,8.752158,25.0,68.0,11.1,15875,33219
Bibb County,7.122016,12.6,82.9,6.6,19918,41770
Blount County,5.130910,13.4,82.0,3.7,21070,45549


En este caso, se puede hacer _slicing_.

In [25]:
df.loc['Alabama':'California',('population')].tail()
#df.loc['Alabama':'California',('population','pop2000')].head()

secundario                   pop2000  pop2010
state      name                              
California Tulare County    368021.0   442179
           Tuolumne County   54501.0    55365
           Ventura County   753197.0   823318
           Yolo County      168660.0   200849
           Yuba County       60219.0    72155

Cuando el `DataFrame` ___está ordenado___, es posible hacer slicing con tuplas. 

In [26]:
df.sort_index().loc[('Alabama','Coosa County'):('Wyoming','Carbon County')].head()

principal                population              money                        \
secundario                  pop2000 pop2010  fed_spend poverty homeownership   
state   name                                                                   
Alabama Coosa County        12202.0   11539   7.839761    16.0          83.7   
        Covington County    37631.0   37765   9.461856    19.0          74.0   
        Crenshaw County     13665.0   13906   9.650295    17.7          67.8   
        Cullman County      77483.0   80406   7.760627    16.7          74.7   
        Dale County         49129.0   50251  25.774791    14.8          61.2   

principal                                             
secundario               multiunit income med_income  
state   name                                          
Alabama Coosa County           1.9  19209      35560  
        Covington County       6.1  19822      33852  
        Crenshaw County        9.2  19793      35140  
        Cullman County         8.5  20284      38567  
        Dale County           13.2  21722      43353

Mediante la función `DataFrame.xs` se puede acceder a los datos especificando el nivel del índice. Esto permite la indexación simple con índices en cualquier nivel.

In [30]:
df.xs('Park County', level=1) # Existen tres condados con el mismo nombre, en diferentes estados.

principal  population              money                                  \
secundario    pop2000 pop2010  fed_spend poverty homeownership multiunit   
state                                                                      
Colorado      14523.0   16206   3.801802     6.2          87.9       1.3   
Montana       15694.0   15636   6.880852    13.6          70.4      11.1   
Wyoming       25786.0   28205  10.813260     9.0          68.7      10.5   

principal                     
secundario income med_income  
state                         
Colorado    31663      64098  
Montana     24717      38830  
Wyoming     26203      46637

Este tipo de acceso también puede hacerse para columnas. 

In [31]:
# El nivel se llama 'secundario' porque anteriormente le hemos llamado así. 
df.xs('pop2000', level='secundario', axis=1).head() 

principal               population
state   name                      
Alabama Autauga County     43671.0
        Baldwin County    140415.0
        Barbour County     29038.0
        Bibb County        20826.0
        Blount County      51024.0

Es posible ordenar el `DataFrame` en función del índice jerárquico. En ese caso, la jerarquía se utiliza también en la ordenación.

In [32]:
df.sort_index(inplace=True)
df.head()

principal              population             money                        \
secundario                pop2000 pop2010 fed_spend poverty homeownership   
state   name                                                                
Alabama Autauga County    43671.0   54571  6.068095    10.6          77.5   
        Baldwin County   140415.0  182265  6.139862    12.2          76.7   
        Barbour County    29038.0   27457  8.752158    25.0          68.0   
        Bibb County       20826.0   22915  7.122016    12.6          82.9   
        Blount County     51024.0   57322  5.130910    13.4          82.0   

principal                                           
secundario             multiunit income med_income  
state   name                                        
Alabama Autauga County       7.2  24568      53255  
        Baldwin County      22.6  26469      50147  
        Barbour County      11.1  15875      33219  
        Bibb County          6.6  19918      41770  
        Blount County        3.7  21070      45549

También se puede ordenar el `DataFrame` en función de un nivel del índice.

In [33]:
df.sort_index(level=1, inplace=True)
df.head()

principal                       population              money          \
secundario                         pop2000 pop2010  fed_spend poverty   
state          name                                                     
South Carolina Abbeville County    26167.0   25417   6.687335    20.7   
Louisiana      Acadia Parish       58861.0   61773   7.444660    20.1   
Virginia       Accomack County     38305.0   33164  13.547069    15.6   
Idaho          Ada County         300904.0  392365   7.957794    10.2   
Iowa           Adair County         8243.0    7682  10.259177    10.6   

principal                                                                  
secundario                      homeownership multiunit income med_income  
state          name                                                        
South Carolina Abbeville County          77.4       7.7  16653      33143  
Louisiana      Acadia Parish             69.8       7.1  18116      37261  
Virginia       Accomack County           74.1       5.6  22766      41372  
Idaho          Ada County                69.6      18.0  27915      55835  
Iowa           Adair County              75.8       9.6  23497      45202

La función `groupby` también puede aplicarse a multi-índices. Acepta un parámetro, denominado `level`, que permite agrupar los datos según el valor del índice en un nivel. 

In [34]:
grupos_df = df.groupby(level=0)
for grupo, datos in grupos_df:
    print(grupo)
    display(datos.head())
    break;    # Solamente el primer grupo.

Alabama


principal              population             money                        \
secundario                pop2000 pop2010 fed_spend poverty homeownership   
state   name                                                                
Alabama Autauga County    43671.0   54571  6.068095    10.6          77.5   
        Baldwin County   140415.0  182265  6.139862    12.2          76.7   
        Barbour County    29038.0   27457  8.752158    25.0          68.0   
        Bibb County       20826.0   22915  7.122016    12.6          82.9   
        Blount County     51024.0   57322  5.130910    13.4          82.0   

principal                                           
secundario             multiunit income med_income  
state   name                                        
Alabama Autauga County       7.2  24568      53255  
        Baldwin County      22.6  26469      50147  
        Barbour County      11.1  15875      33219  
        Bibb County          6.6  19918      41770  
        Blount County        3.7  21070      45549

Otro ejemplo. Agrupa por nombre de condado, y muestra las columnas que corresponden al índice _population_.

In [37]:
grupos_df = df.groupby(level=1)
for grupo, datos in grupos_df:
    print(grupo)
    display(datos[('population')].head())
    break;    # Solamente el primer grupo

Abbeville County


,secundario,pop2000,pop2010
state,name,,
South Carolina,Abbeville County,26167.0,25417


Por último, a veces es más cómodo mostrar los índices como tuplas. Es posible hacerlo fijando la opción `display.multi_sparse` de Pandas a `False`. 

In [38]:
pd.set_option('display.multi_sparse', False)
df.sort_index().head()

,principal,population,population,money,money,money,money,money,money
,secundario,pop2000,pop2010,fed_spend,poverty,homeownership,multiunit,income,med_income
state,name,,,,,,,,
Alabama,Autauga County,43671.0,54571,6.068095,10.6,77.5,7.2,24568,53255
Alabama,Baldwin County,140415.0,182265,6.139862,12.2,76.7,22.6,26469,50147
Alabama,Barbour County,29038.0,27457,8.752158,25.0,68.0,11.1,15875,33219
Alabama,Bibb County,20826.0,22915,7.122016,12.6,82.9,6.6,19918,41770
Alabama,Blount County,51024.0,57322,5.130910,13.4,82.0,3.7,21070,45549


In [39]:
# Se restaura
pd.set_option('display.multi_sparse', True)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section10"></a>
# <font color="#004D7F"> 10. Combinación de Dataframes</font>
<br>


Una de las funcionalidades más interesantes de _Pandas_ es la relativa a la agregación de datos de distintas fuentes. La documentación oficial de la librería ilustra con ejemplos la mayoría de casos de uso ([documentación](https://pandas.pydata.org/pandas-docs/stable/merging.html)).

Para ilustrar los ejemplos de este tutorial, se utilizarán estos tres `DataFrame`.


In [40]:
pos1_df = pd.DataFrame([('Diego Costa','Delantero', 'Brasil'),
                        ('Sergio Ramos', 'Defensa', 'España'),
                        ('Gerard Piqué', 'Defensa', 'España'),
                        ('Cristiano Ronaldo', 'Delantero','Portugal')],
                       columns = ['Nombre','Posición', 'País'])

pos2_df = pd.DataFrame([('Leo Messi','Delantero', 'Argentina'),
                        ('Luca Modric', 'Centrocampista', 'Croacia'),
                        ('Saúl Ñíguez', 'Centrocampista', 'España'),
                        ('Kareem Benzema', 'Delantero','Francia')],
                       columns = ['Nombre','Posición', 'País'])

eqp_df = pd.DataFrame([('Diego Costa', 'Atlético de Madrid', 'España'),
                       ('Cristiano Ronaldo','Juventus', 'Italia'),
                       ('Leo Messi','FC Barcelona', 'España'),
                       ('Koke', 'Atlético de Madrid', 'España')],
                    columns = ['Nombre','Equipo', 'País'])
                         
display(pos1_df)
  
display(pos2_df)
    
display(eqp_df)

,Nombre,Posición,País
0,Diego Costa,Delantero,Brasil
1,Sergio Ramos,Defensa,España
2,Gerard Piqué,Defensa,España
3,Cristiano Ronaldo,Delantero,Portugal


,Nombre,Posición,País
0,Leo Messi,Delantero,Argentina
1,Luca Modric,Centrocampista,Croacia
2,Saúl Ñíguez,Centrocampista,España
3,Kareem Benzema,Delantero,Francia


,Nombre,Equipo,País
0,Diego Costa,Atlético de Madrid,España
1,Cristiano Ronaldo,Juventus,Italia
2,Leo Messi,FC Barcelona,España
3,Koke,Atlético de Madrid,España


<a id="section101"></a> 
## <font color="#004D7F"><font face="monospace"> append()</font></font>
    
Es la función más sencilla. Permite añadir a un `DataFrame` las filas de otro u otros `Dataframe`. Como resultado, genera un nuevo `DataFrame`.

In [41]:
display(pos1_df.set_index("Nombre").append(pos2_df.set_index("Nombre")))

,Posición,País
Nombre,,
Diego Costa,Delantero,Brasil
Sergio Ramos,Defensa,España
Gerard Piqué,Defensa,España
Cristiano Ronaldo,Delantero,Portugal
Leo Messi,Delantero,Argentina
Luca Modric,Centrocampista,Croacia
Saúl Ñíguez,Centrocampista,España
Kareem Benzema,Delantero,Francia


`append()` toma un parámetro, denominado `ignore_index` que permite crear un nuevo índice (numérico) e ignorar el de los `DataFrames` originales. 

In [42]:
display(pos1_df.append(pos2_df, ignore_index=True))

,Nombre,Posición,País
0,Diego Costa,Delantero,Brasil
1,Sergio Ramos,Defensa,España
2,Gerard Piqué,Defensa,España
3,Cristiano Ronaldo,Delantero,Portugal
4,Leo Messi,Delantero,Argentina
5,Luca Modric,Centrocampista,Croacia
6,Saúl Ñíguez,Centrocampista,España
7,Kareem Benzema,Delantero,Francia


<div class="alert alert-block alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: La función `append()` es en realidad un caso específico de la función más general `concat()`, que se verá a continuación.
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section102"></a> 
## <font color="#004D7F"><font face="monospace"> concat()</font></font>

Esta función implementa la concatenación de _DataFrames_. Puede hacerse tanto a nivel de filas (similar a `append()`) como de columnas. Toma varios parámetros [(documentación)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html). Los más importantes, además de la lista de _DataFrames_ que se han de concatenar, son:

* `axis`. Determina el eje a lo largo del cual se concatenan los datos, y puede tomar los valores 0 (filas) y 1 (columnas).
* `join`. Determina si se considera la unión (`outer`) o la intersección (`inner`) de elementos en el otro eje. 
* `keys`. Es un vector de claves. Si se utiliza, crea un multi-índice, y utiliza estas claves en el primer nivel para marcar el `DataFrame` original en el resultante. 

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Existe otro parámetro, denominado `join_axes`, que permite especificar qué elementos se incluyen (se usa en lugar de `join`) en el nuevo `DataFrame`. En la última versión de Pandas está marcado como obsoleto _(Deprecated)_.
</div>


La siguiente llamada es equivalente a `append()`. Por defecto lleva a cabo la concatenación a nivel de filas, y une las columnas de ambos `DataFrame`.

In [45]:
# pd.concat([pos1_df, pos2_df])
pd.concat([pos1_df, pos2_df], join="outer", axis=0)       # Es equivalente

,Nombre,Posición,País
0,Diego Costa,Delantero,Brasil
1,Sergio Ramos,Defensa,España
2,Gerard Piqué,Defensa,España
3,Cristiano Ronaldo,Delantero,Portugal
0,Leo Messi,Delantero,Argentina
1,Luca Modric,Centrocampista,Croacia
2,Saúl Ñíguez,Centrocampista,España
3,Kareem Benzema,Delantero,Francia


El parámetro `join` determina qué conjunto de índices (__en el eje que no se concatena__) se incluye en el `DataFrame` resultado. El siguiente ejemplo concatena las filas de ambos `DataFrame`, y solamente incluye, mediante `join=inner` las columnas que aparecen en ambos.

In [48]:
#display(pos1_df)
#display(eqp_df)

pd.concat([pos1_df, eqp_df], join="inner", axis=0)

,Nombre,País
0,Diego Costa,Brasil
1,Sergio Ramos,España
2,Gerard Piqué,España
3,Cristiano Ronaldo,Portugal
0,Diego Costa,España
1,Cristiano Ronaldo,Italia
2,Leo Messi,España
3,Koke,España


Cuando se unen _DataFrames_ con distintas columnas, y se usa `join=outer`, los valores indeterminados se fijan a _NaN_ en el nuevo _DataFrame_. Este código, además, añade una clave que permite identificar el origen de los datos. 

In [54]:
#display(pos1_df)
#display(eqp_df)

pd.concat([pos1_df, eqp_df], join="outer", axis=0, keys=["jugadores", "Equipos"], sort=False)
#pd.concat([pos1_df, eqp_df], join="outer", axis=0, keys=["jugadores", "Equipos"])

Nombre   Posición      País              Equipo
jugadores 0        Diego Costa  Delantero    Brasil                 NaN
          1       Sergio Ramos    Defensa    España                 NaN
          2       Gerard Piqué    Defensa    España                 NaN
          3  Cristiano Ronaldo  Delantero  Portugal                 NaN
Equipos   0        Diego Costa        NaN    España  Atlético de Madrid
          1  Cristiano Ronaldo        NaN    Italia            Juventus
          2          Leo Messi        NaN    España        FC Barcelona
          3               Koke        NaN    España  Atlético de Madrid

La elección de `axis=1` permite concatenar las columnas. En este ejemplo, indicamos que solamente se consideren aquellas filas cuyo índice aparece en ambos `DataFrame` mediante `join=inner`.

In [55]:
# Previamente, establecemos el nombre del jugador como índice. 
cp_pos1_df=pos1_df.set_index('Nombre')
cp_pos2_df=pos2_df.set_index('Nombre')
cp_eqp_df = eqp_df.set_index('Nombre'); # Punto y coma para que no se muestre la salida

# display(cp_pos1_df)
# display(cp_eqp_df)

In [56]:
pd.concat([cp_pos1_df, cp_eqp_df], axis=1, join='inner')

,Posición,País,Equipo,País
Nombre,,,,
Diego Costa,Delantero,Brasil,Atlético de Madrid,España
Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia


Si se especifica `join=outer` se cogen todos los jugadores. 

In [57]:
pd.concat([cp_pos1_df, cp_eqp_df], axis=1, join='outer', sort=False)

,Posición,País,Equipo,País
Diego Costa,Delantero,Brasil,Atlético de Madrid,España
Sergio Ramos,Defensa,España,NaN,NaN
Gerard Piqué,Defensa,España,NaN,NaN
Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia
Leo Messi,NaN,NaN,FC Barcelona,España
Koke,NaN,NaN,Atlético de Madrid,España


En lugar de `join`, es posible determinar qué índices se incluyen mediante `join_axes`. El siguiente ejemplo incluye todas las filas del primer `DataFrame`.

In [58]:
# Deprecated
# pd.concat([cp_pos1_df, cp_eqp_df], axis=1, join_axes=[cp_pos1_df.index])

TypeError: concat() got an unexpected keyword argument 'join_axes'

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section103"></a> 
## <font color="#004D7F"><font face="monospace">merge()</font></font>

<br>

La función `merge()` permite unir las columnas de ___dos___ `DataFrame`. A diferencia de `concat()`, permite especificar la columna a partir de la cual se lleva a cabo esa unión. Acepta numerosos argumentos ([documentación](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)) que rigen la unión. Algunos de los más importantes son:

* `left`, `right`. Son argumentos posicionales que se refieren a los dos `DataFrame` que son unidos. 
* `left_index`, `right_index`. Determinan si los índices respectivos se usan como claves de unión.
* `on`, `left_on`, `right_on`. Determinan qué columnas (si no se usan índices) son utilizadas como claves de unión. `on` se utiliza cuando las columnas aparecen en ambos `DataFrame`.
* `how`. Determina qué filas se incluyen en la unión. Puede tomar los valores `left`, `right`, `outer`, e `inner` según se consideren, respectivamente, los índices del primer `DataFrame`, del segundo, la unión, o la intersección de ambos.  

Además, admite otros parámetros de utilidad a la hora de presentar el conjunto de datos resultante de la unión.

* `suffixes`. Es una lista de `Strings` (dos). Cuando existen columnas comunes en ambos `DataFrame`, y no son utilizadas como clave de unión, permite identificarlas en el `DataFrame` resultante. Para ello, añade cada `String` al nombre de la columna correspondiente según incluya los valores de uno u otro `DataFrame`.  
* `indicator`. Añade una columna, denominada `_merge` con información sobre el origen de cada fila (un `DataFrame` concreto o los dos).
* `validate`. Es un `String` que permite determinar si se cumple una determinada relación entre las claves de unión. Puede tomar los valores `1:1`, `1:m`, `m:1` y `m:m`.

En la siguiente celda se lleva a cabo la unión entre los dos `DataFrame` definidos anteriormente en función del nombre del jugador, y considerando la unión de todas las filas. Como la columna _País_ aparece en ambos `DataFrame`, se añade también un sufijo para determinar la correspondencia en el `DataFrame` resultante.

In [61]:
print('DataFrame izquierdo:')
display(pos1_df)
print('DataFrame derecho:')
display(eqp_df)
print('Unión:')
#pd.merge(pos1_df,eqp_df, how='outer', left_on='Nombre', right_on='Nombre', suffixes=['_jug','_equ'])
pd.merge(pos1_df,eqp_df, how='outer', on='Nombre', suffixes=['_jug','_equ'])  # Equivalente

DataFrame izquierdo:


,Nombre,Posición,País
0,Diego Costa,Delantero,Brasil
1,Sergio Ramos,Defensa,España
2,Gerard Piqué,Defensa,España
3,Cristiano Ronaldo,Delantero,Portugal


DataFrame derecho:


,Nombre,Equipo,País
0,Diego Costa,Atlético de Madrid,España
1,Cristiano Ronaldo,Juventus,Italia
2,Leo Messi,FC Barcelona,España
3,Koke,Atlético de Madrid,España


Unión:


,Nombre,Posición,País_jug,Equipo,País_equ
0,Diego Costa,Delantero,Brasil,Atlético de Madrid,España
1,Sergio Ramos,Defensa,España,NaN,NaN
2,Gerard Piqué,Defensa,España,NaN,NaN
3,Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia
4,Leo Messi,NaN,NaN,FC Barcelona,España
5,Koke,NaN,NaN,Atlético de Madrid,España


En este caso, suponemos que los `DataFrame` están indexados según el nombre del jugador. Además, añadimos un indicador, que permite determinar el origen de cada entrada. 

In [62]:
pd.merge(pos1_df.set_index('Nombre'), eqp_df.set_index('Nombre'), how='outer', 
         left_index=True, right_index=True, suffixes=['_jug','_equ'] , indicator=True)

,Posición,País_jug,Equipo,País_equ,_merge
Nombre,,,,,
Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia,both
Diego Costa,Delantero,Brasil,Atlético de Madrid,España,both
Gerard Piqué,Defensa,España,NaN,NaN,left_only
Koke,NaN,NaN,Atlético de Madrid,España,right_only
Leo Messi,NaN,NaN,FC Barcelona,España,right_only
Sergio Ramos,Defensa,España,NaN,NaN,left_only


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section104"></a> 

## <font color="#004D7F"><font face="monospace"> join()</font></font>

Es similar a `merge()`, aunque permite unir las columnas de __varios__ `DataFrame` y utiliza solo algunos parámetros. Por defecto utiliza los índices como clave de unión y el conjunto de elementos del `DataFrame` de la izquierda, es decir, `how=left` ([documentación](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html)).

In [63]:
pos1_df.set_index('Nombre', inplace=True)
pos2_df.set_index('Nombre', inplace=True)
eqp_df.set_index('Nombre', inplace=True)

# display(pos1_df)
# display(pos2_df) 
# display(eqp_df)

Une dos _DataFrame_ y añade un sufijo a las columnas comunes. 

In [64]:
pos1_df.join(eqp_df, lsuffix='_jug', rsuffix='_equ')

,Posición,País_jug,Equipo,País_equ
Nombre,,,,
Diego Costa,Delantero,Brasil,Atlético de Madrid,España
Sergio Ramos,Defensa,España,NaN,NaN
Gerard Piqué,Defensa,España,NaN,NaN
Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia


Hace lo mismo, pero utilizando `merge`.

In [65]:
pd.merge(pos1_df, eqp_df, left_index=True, right_index=True, how='left', suffixes=['_jug','_equ'], sort=False)

,Posición,País_jug,Equipo,País_equ
Nombre,,,,
Diego Costa,Delantero,Brasil,Atlético de Madrid,España
Sergio Ramos,Defensa,España,NaN,NaN
Gerard Piqué,Defensa,España,NaN,NaN
Cristiano Ronaldo,Delantero,Portugal,Juventus,Italia


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section11"></a>
# <font color="#004D7F"> 11. Tablas dinámicas y reestructuración </font>

<br>
<a id="section111"></a>

## <font color="#004D7F">Creación de tablas dinámicas</font>
</font>

### <font color="#004D7F"> <font face="monospace">pivot() </font></font>

En su versión más sencilla, una tabla dinámica es una tabla que se construye a partir de tres columnas de otra dada, de modo que una de las columnas actúa como índice, otra como columnas, y otra como valores.  La función `DataFrame.pivot` permite construir estas tablas.  

In [66]:
ventas = [('Albacete', 22.5, 'Queso', 25),
         ('Villarrobledo', 21, 'Queso', 23.5),
         ('Albacete', 50, 'Jamón', 60),
         ('La Roda', 20.0, 'Aceite', 25),
         ('Villarrobledo', 47, 'Jamón', 55),
         ('La Roda', 50.0, 'Jamón', 55)]

columnas = ['Localidad', 'Precio', 'Producto', 'P.V.P']
indice =['Tienda 1', 'Tienda 2', 'Tienda 1', 'Tienda 3', 'Tienda 2', 'Tienda 3']

df_compras = pd.DataFrame(ventas, columns=columnas, index=indice)
df_compras

,Localidad,Precio,Producto,P.V.P
Tienda 1,Albacete,22.5,Queso,25.0
Tienda 2,Villarrobledo,21.0,Queso,23.5
Tienda 1,Albacete,50.0,Jamón,60.0
Tienda 3,La Roda,20.0,Aceite,25.0
Tienda 2,Villarrobledo,47.0,Jamón,55.0
Tienda 3,La Roda,50.0,Jamón,55.0


In [67]:
df_compras.pivot(index='Producto', columns='Localidad', values='Precio')

Localidad,Albacete,La Roda,Villarrobledo
Producto,,,
Aceite,NaN,20.0,NaN
Jamón,50.0,50.0,47.0
Queso,22.5,NaN,21.0


Es posible utilizar varios valores. En ese caso, la tabla creada utiliza un multi-índice para las columnas. Por otra parte, si no se especifican valores, se utilizan todas las columnas. 

In [68]:
#df_compras.pivot(index='Producto', columns='Localidad')       # Equivalentes
df_compras.pivot(index='Producto', columns='Localidad', values=['Precio','P.V.P'])  

Precio                          P.V.P                      
Localidad Albacete La Roda Villarrobledo Albacete La Roda Villarrobledo
Producto                                                               
Aceite         NaN    20.0           NaN      NaN    25.0           NaN
Jamón         50.0    50.0          47.0     60.0    55.0          55.0
Queso         22.5     NaN          21.0     25.0     NaN          23.5

La función `DataFrame.pivot` asume que cada par de valores _(índice, columna)_ aparece solo una vez en la tabla original. En caso contrarío, devuelve un error. 

In [69]:
ventas = [('Albacete', 22.5, 'Queso', 25),
         ('Villarrobledo', 21, 'Queso', 23.5),
         ('Albacete', 50, 'Jamón', 60),
         ('La Roda', 20.0, 'Aceite', 25),
         ('Villarrobledo', 47, 'Jamón', 55),
         ('La Roda', 50.0, 'Jamón', 55),
         ('Villarrobledo', 23, 'Queso', 22.5)] # Esta es la línea añadida

columnas = ['Localidad', 'Precio', 'Producto', 'P.V.P']
indice =['Tienda 1', 'Tienda 2', 'Tienda 1', 'Tienda 3', 'Tienda 2', 'Tienda 3', 'Tienda 4']

df_compras = pd.DataFrame.from_records(ventas, columns=columnas, index=indice)
df_compras

,Localidad,Precio,Producto,P.V.P
Tienda 1,Albacete,22.5,Queso,25.0
Tienda 2,Villarrobledo,21.0,Queso,23.5
Tienda 1,Albacete,50.0,Jamón,60.0
Tienda 3,La Roda,20.0,Aceite,25.0
Tienda 2,Villarrobledo,47.0,Jamón,55.0
Tienda 3,La Roda,50.0,Jamón,55.0
Tienda 4,Villarrobledo,23.0,Queso,22.5


In [70]:
# Devolvería error porque (Queso, Villarrobledo) aparece dos veces.
# df_compras.pivot(index='Producto', columns='Localidad', values='Precio') 

ValueError: Index contains duplicate entries, cannot reshape

### <font color="#004D7F"> <font face="monospace">pivot_table() </font></font>

La función `pivot_table` permite construir una tabla dinámica a partir de los datos de un `DataFrame`. Es parecida a la anterior, pero utiliza una __función de agregación__ para agregar datos correspondientes al mismo par fila/columna (situación anterior). Toma principalmente cuatro parámetros [(documentación)](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html):

* `index`/`columns`. La columna o columnas cuyos valores serán utilizados como índices/columnas en la nueva tabla. 
* `values`. Los valores de interés, sobre los que se lleva a cabo la agregación.
* `aggfunc`. La función o funciones de agregación.

Para ilustrar el uso de esta función se almacena el conjunto de datos `Titanic.csv` en un _DataFrame_, y se crea una columna con una etiqueta relativa a la franja de edad, que divide a los pasajeros en tres según sean menores de 18 (joven), mayores de 65 (anciano), o el resto (adulto).

In [71]:
def franja_edad(edad):
    if edad<18: return 'joven'
    if edad>65: return 'anciano'
    return 'adulto'
    
df_titanic = pd.read_csv('data/Titanic.csv', sep=',', skiprows=1, index_col=1,
                         names=['ID','Nombre','Clase','Edad','Sexo','Superviviente','Código (Sexo)'])

df_titanic['FranjaEdad']=df_titanic['Edad'].map(franja_edad)
del df_titanic['ID']
df_titanic.head()

,Clase,Edad,Sexo,Superviviente,Código (Sexo),FranjaEdad
Nombre,,,,,,
"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1,adulto
"Allison, Miss Helen Loraine",1st,2.00,female,0,1,joven
"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0,adulto
"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1,adulto
"Allison, Master Hudson Trevor",1st,0.92,male,1,0,joven


La siguiente función muestra la supervivencia media en función de los campos _Sexo_ y _Clase_. (La función `dropna` elimina los registros vacíos).

In [72]:
pt = (df_titanic.pivot_table(index='Clase', columns='Sexo', values='Superviviente', aggfunc=np.mean)
                .dropna())
pt

Sexo,female,male
Clase,,
1st,0.937063,0.329609
2nd,0.878505,0.145349
3rd,0.377358,0.116232


En este otro ejemplo, se utilizan como columnas tanto el sexo como la franja de edad para contar los supervivientes (`aggfunc=np.sum`). Además, se sustituyen los valores que no están presentes por `--`, y se añaden márgenes con el total.

In [73]:
df_titanic.pivot_table(index='Clase', columns=['Sexo', 'FranjaEdad'], values=['Superviviente'], 
                       aggfunc=np.sum, fill_value='--', margins=True, margins_name='Total')

Superviviente                                         
Sexo              female                 male               Total
FranjaEdad        adulto anciano joven adulto anciano joven      
Clase                                                            
*                     --      --    --      0      --    --     0
1st                  127       1     6     53       0     6   193
2nd                   80      --    14     14       0    11   119
3rd                   66      --    14     51      --     7   138
Total                273       1    34    118       0    24   450

También se pueden especificar varias columnas como valores, o varias funciones de agregación.

In [74]:
df_titanic.pivot_table(index='Clase', columns=['Sexo'], values=['Superviviente','Edad'], 
                       fill_value=0, aggfunc={'Superviviente':[np.mean, np.sum], 'Edad':[np.mean]}).round(2)

Edad        Superviviente                  
        mean                 mean          sum     
Sexo  female   male        female  male female male
Clase                                              
*       0.00   0.00          0.00  0.00      0    0
1st    37.77  41.20          0.94  0.33    134   59
2nd    27.39  28.91          0.88  0.15     94   25
3rd    22.78  26.36          0.38  0.12     80   58

### <font color="#004D7F"> <font face="monospace">crosstab() </font></font>

La función `pandas.crosstab()` es parecida a la anterior. En lugar de un _DataFrame_ toma como argumentos las secuencias de índices, columnas y valores, por separado. Además, por defecto usa `aggfunc=len` y los valores perdidos los sustituye por 0.

In [75]:
sexo = df_titanic.Sexo.values[:10]
edad = df_titanic.FranjaEdad.values[:10]
display(sexo)
display(edad)

pd.crosstab(sexo, edad, rownames=['Sexo'], colnames=['Edad'], margins=True)
#pd.crosstab(df_titanic['Sexo'], df_titanic['FranjaEdad'], rownames=['Sexo'], colnames=['Edad'], margins=True)

array(['female', 'female', 'male', 'female', 'male', 'male', 'female',
       'male', 'female', 'male'], dtype=object)

array(['adulto', 'joven', 'adulto', 'adulto', 'joven', 'adulto', 'adulto',
       'adulto', 'adulto', 'anciano'], dtype=object)

Edad,adulto,anciano,joven,All
Sexo,,,,
female,4,0,1,5
male,3,1,1,5
All,7,1,2,10


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<a id="section112"></a>

## <font color="#004D7F">Reestructuración de tablas</font>
</font>

### <font color="#004D7F"> <font face="monospace">stack() </font></font>

Es posible llevar a cabo el proceso inverso al descrito anteriormente. Ese proceso se denomina __apilar__ y consiste en utilizar el producto cartesiano del índice y las columnas como índice en las filas. Para ello, se utiliza la función `stack()`. El resultado es un `DataFrame` indexado mediante un multi-índice.

In [76]:
import pandas as pd
df_precios = pd.DataFrame({'Casa':[10,20,30], 'Coche':[40,50,60], 'Transporte':[1,2,3]},
                         index = ['Madrid', 'Barcelona', 'Paris'])
df_precios

,Casa,Coche,Transporte
Madrid,10,40,1
Barcelona,20,50,2
Paris,30,60,3


In [77]:
df_precios_stack = df_precios.stack()
df_precios_stack 

Madrid     Casa          10
           Coche         40
           Transporte     1
Barcelona  Casa          20
           Coche         50
           Transporte     2
Paris      Casa          30
           Coche         60
           Transporte     3
dtype: int64

### <font color="#004D7F"> <font face="monospace">unstack() </font></font>

La función `unstack()` es inversa a la anterior. Utiliza el nivel inferior del índice de filas como índice de columnas. 

In [78]:
df_precios_stack.unstack()

,Casa,Coche,Transporte
Madrid,10,40,1
Barcelona,20,50,2
Paris,30,60,3


### <font color="#004D7F"> <font face="monospace">melt() </font></font>

Otra posibilidad es `melt()`. Esta función es parecida a `stack()`, pero utiliza los nombres de varias columnas como valores en otra nueva (en lugar de un índice como `stack()`). Toma tres parámetros principales. 

* `id_vars`. Lista de columnas de referencia. 
* `value_vars`. Lista de columnas que son despivotadas. 
* `var_name` y `value_name`. Nombre de las columnas con los nombres de las variables, y los valores correspondientes. 

In [79]:
df_precios.reset_index(inplace=True)
df_precios.columns = ['Ciudad','Casa','Coche','Transporte']
df_precios

,Ciudad,Casa,Coche,Transporte
0,Madrid,10,40,1
1,Barcelona,20,50,2
2,Paris,30,60,3


In [80]:
td = df_precios.melt(id_vars='Ciudad', value_vars=['Casa','Coche'], var_name='Concepto', value_name='Precio')
td

,Ciudad,Concepto,Precio
0,Madrid,Casa,10
1,Barcelona,Casa,20
2,Paris,Casa,30
3,Madrid,Coche,40
4,Barcelona,Coche,50
5,Paris,Coche,60


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>